In [1]:
import tensorflow as tf

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=6024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p "kaggle"

In [ ]:
import shutil

shutil.copy('kaggle.json','kaggle/')

In [ ]:
import os

# Specify the file path and the desired permissions
file_path = 'kaggle/kaggle.json'
permissions = 600  # Example: Read and write permissions for the owner only

# Change the file permissions using the os module
os.chmod(file_path, permissions)


In [ ]:
! kaggle datasets download nikhilroxtomar/person-segmentation

In [ ]:
! kaggle datasets download -d tapakah68/supervisely-filtered-segmentation-person-dataset

In [ ]:
! kaggle datasets download -d kapitanov/easyportrait

In [22]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
import matplotlib.pyplot as plt
from tqdm import tqdm
import urllib
import IPython

convert_to_mask('ds1_pexels-photo-175697.png')

In [23]:
"""Model Bulding Code"""

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    s5, p5 = encoder_block(p4, 1024)
    b1 = conv_block(p5, 2048)

    """ Decoder """
    d1 = decoder_block(b1, s5, 1024)
    d2 = decoder_block(d1, s4, 512)
    d3 = decoder_block(d2, s3, 256)
    d4 = decoder_block(d3, s2, 128)
    d5 = decoder_block(d4, s1, 64)

    """ Output """
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d5)
    return Model(inputs, outputs, name="U-Net")

In [41]:
"""Dataset preparation Code"""

def augment(image, mask, s):
    flip = layers.RandomFlip("horizontal_and_vertical", seed=s) # or "horizontal", "vertical"
    rotate = tf.keras.layers.RandomRotation(0.3, seed=s)
    zoom = tf.keras.layers.RandomZoom(0.3,seed=s)
    lst = [flip, rotate, zoom]
    for i in range(len(lst)):
        image = lst[i](image)
        mask = lst[i](mask)
        
    return image, mask

def load_data(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, "images/*")))
    masks = sorted(glob(os.path.join(dataset_path, "masks_/*")))
#     for i in range(2):
#         print(images[i])
#         print(masks[i])
    train_x, test_x = train_test_split(images, test_size=0.1, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=0.1, random_state=42)
    return (train_x, train_y), (test_x, test_y)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    # (256, 256, 3)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path = image_path.decode()
        mask_path = mask_path.decode()
        x = read_image(image_path)
        y = read_mask(mask_path)
        return x, y
    image, mask = tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    seed = np.random.randint(500)
    image, mask = augment(image, mask, seed)    
    image.set_shape([256, 256, 3])
    mask.set_shape([256, 256, 1])
    return image, mask

def tf_dataset(images, masks, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.shuffle(buffer_size=30000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

In [42]:
""" Hyperparameters """
dataset_path = "Backgrougd Removal/Background Dataset/"
input_shape = (256, 256, 3)
batch_size = 8
epochs = 60
lr = 1e-3
model_path = "./RamBG Data/Models/unet_july_15.h5"
csv_path = "./RamBG Data/data_july_15.csv"

In [43]:
import os
os.listdir("Backgrougd Removal/")

['Background Dataset',
 'Background Dataset - Shortcut.lnk',
 'EasyPortrait',
 'easyportrait.zip',
 'masks',
 'people_segmentation',
 'person-segmentation.zip',
 'supervisely-filtered-segmentation-person-dataset.zip',
 'supervisely_person_clean_2667_img',
 'Test']

In [44]:
""" Loading the dataset """
(train_x, train_y), (test_x, test_y) = load_data(dataset_path)

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(test_x, test_y, batch=batch_size)

In [45]:
len(valid_dataset)

330

In [46]:
train_dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

In [47]:
train_dataset = train_dataset.skip(700)

In [48]:
len(train_dataset)

2264

In [37]:
""" Model """
model = build_unet(input_shape)
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision()
    ]
)

callbacks = [
    ModelCheckpoint(model_path, monitor="val_loss", verbose=1, save_best_only=False),
    ReduceLROnPlateau(monitor="val_loss", patience=4, factor=0.1, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

In [38]:
train_steps = len(train_x)//batch_size - 700
if len(train_x) % batch_size != 0:
    train_steps += 1

valid_steps = len(test_x)//batch_size
if len(test_x) % batch_size != 0:
    valid_steps += 1


In [39]:
valid_steps

330

In [40]:
train_steps

2264

In [41]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs = epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

Epoch 1/60
2264/2264 [==============================] - ETA: 0s - loss: 0.2784 - mean_io_u_1: 0.2883 - recall_1: 0.8522 - precision_1: 0.8619
Epoch 1: saving model to ./RamBG Data/Models\unet_july_15.h5
2264/2264 [==============================] - 1399s 506ms/step - loss: 0.2784 - mean_io_u_1: 0.2883 - recall_1: 0.8522 - precision_1: 0.8619 - val_loss: 0.2294 - val_mean_io_u_1: 0.2849 - val_recall_1: 0.8559 - val_precision_1: 0.9123 - lr: 0.0010
Epoch 2/60
2264/2264 [==============================] - ETA: 0s - loss: 0.1570 - mean_io_u_1: 0.2886 - recall_1: 0.9258 - precision_1: 0.9275
Epoch 2: saving model to ./RamBG Data/Models\unet_july_15.h5
2264/2264 [==============================] - 1291s 468ms/step - loss: 0.1570 - mean_io_u_1: 0.2886 - recall_1: 0.9258 - precision_1: 0.9275 - val_loss: 0.1473 - val_mean_io_u_1: 0.2858 - val_recall_1: 0.9513 - val_precision_1: 0.9242 - lr: 0.0010
Epoch 3/60
2264/2264 [==============================] - ETA: 0s - loss: 0.1171 - mean_io_u_1: 0.2886

In [4]:
model = tf.keras.models.load_model("RamBG Data/Models/unet_july_15.h5")

In [5]:
A= cv2.imread("E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\download (1).jpeg")

In [6]:
def convert_(s1):
    return s1.replace('\\','/')

In [7]:
convert_("E:\Photos\Archive\E40DFFDF-2B49-478E-BC1C-DC4CF4A59CE2.JPG")

'E:/Photos/Archive/E40DFFDF-2B49-478E-BC1C-DC4CF4A59CE2.JPG'

In [14]:
test_images = [
#     convert_(r"E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\images\ds1_family-toddler-hapy-happy-160688.png")
#     convert_("E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\Test1.jpg"),
#     convert_("E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\Test2.jpg"),
#     convert_("E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\Test3.jpg"),
#     convert_("E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\Test4.jpg")
    convert_("E:\College\Term2\AI ML Lab\Backgrougd Removal\Background Dataset\WhatsApp Image_V.jpg")
]

In [15]:
test_images

['E:/College/Term2/AI ML Lab/Backgrougd Removal/Background Dataset/WhatsApp Image_V.jpg']

In [16]:
for path in tqdm(test_images, total=len(test_images)):
    # req = path
    # imgarr = np.asarray(bytearray(req.read()), dtype=np.uint8)

    # x = cv2.imdecode(imgarr, -1)

    x = cv2.imread(path, cv2.IMREAD_COLOR)
    original_image = x
#     print(x)
    h, w, _ = x.shape

    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)

    pred_mask = model.predict(x)[0]
    pred_mask = cv2.resize(pred_mask, (w, h))
    pred_mask = np.expand_dims(pred_mask, axis=-1)
    pred_mask = pred_mask > 0.5

    background_mask = np.abs(1- pred_mask)

    masked_image = original_image * pred_mask

    background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1)
    background_mask = background_mask * [0, 0, 0]

    masked_image = masked_image + background_mask
    name = path.split("/")[-1]
    cv2.imwrite(f"RamBG Data/Outputs/{name}.png", masked_image)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.32it/s]
